In [ ]:
import numpy as np
import pandas as pd
from tqdm.notebook import tqdm
import torch
import torchvision
import torchvision.transforms as T
from collections import defaultdict, deque
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor
from torchvision.models.detection import FasterRCNN
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor
from torchvision.models.detection import FasterRCNN
from torchvision.models.detection.rpn import AnchorGenerator
import ast
from torch.utils.data import DataLoader, Dataset
from torch.utils.data.sampler import SequentialSampler
import albumentations as A
from albumentations.pytorch.transforms import ToTensorV2

import cv2
import os,sys,matplotlib,re
from PIL import Image
from skimage import exposure
from pydicom.pixel_data_handlers.util import apply_voi_lut
import matplotlib.pyplot as plt
import matplotlib.image as immg

import warnings
warnings.filterwarnings("ignore")

In [ ]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

In [ ]:
# Acá debemos escribir la ruta de la carpeta que contiene a las imágenes.
path = "../input/wheat-train512x512/" 

In [ ]:
# Acá debemos escribir la ruta del archivo csv.
df = pd.read_csv('../input/global-wheat-detection/train.csv')
df.head()

La columna bbox contiene strings, las cuales son listas coercionadas al tipo string.

La función *literal_eval* del paquete *ast* nos permite convertir estas strings de vuelta en listas.

&nbsp;

Ejemplo:

In [ ]:
print(df["bbox"][0])
type(df["bbox"][0])

In [ ]:
print(ast.literal_eval(df["bbox"][0]))
type(ast.literal_eval(df["bbox"][0]))

Estos números describen a la bounding box. La menor coordenada  x  de la bounding box es 834. La menor coordenada  y  de la bounding box es 222. Es decir, la esquina inferior izquierda de la bounding box tiene coordenadas  (834,222) . El ancho de la bounding box es 56, y su alto es 36.

En lugar de tener en una sola columna los datos de la bounding box, vamos a darle su propia columna a cada uno de los cuatro datos que describen a la bounding box.

In [ ]:
df['x_min'] = np.nan
df['y_min'] = np.nan
df['w'] = np.nan
df['h'] = np.nan

df[['x_min', 'y_min', 'w', 'h']] = np.stack(df['bbox'].apply(lambda x: ast.literal_eval(x)))

# Habiendo hecho esto podemos prescindir de la columna bbox.
df.drop(columns=['bbox'], inplace=True)

Luego de esta operación, así es como queda nuestro data frame:

In [ ]:
df.head()

# Visualización de imágenes de entrenamiento

Cada fila de nuestro data frame corresponde a una bounding box. Nótese que varias filas pueden compartir la misma image_id.

Si eso ocurre es porque asociada a una misma imagen hay varias bounding boxes.

Hagamos el ejercicio de visualizar una imagen y dibujar todas las bounding boxes que aparecen en ella.

&nbsp;

**Un detalle: nosotros hemos hecho un resizing a las imágenes.** Originalmente las imágenes eran de dimensiones 1024 x 1024 x 3. **Ahora son de dimensiones 512 x 512 x 3. Hemos hecho esto para acelerar el proceso de entrenamiento.**

**Para que las bounding boxes quepan en nuestras imágenes reescaladas vamos a tener que editar las columnas de nuestro data frame.**



In [ ]:
# En primer lugar hacemos el reescalamiento de las bounding boxes.
df["x_min"] = df["x_min"].astype(np.float)*512/1024
df["y_min"] = df["y_min"].astype(np.float)*512/1024
df["h"] = df["h"].astype(np.float)*512/1024
df["w"] = df["w"].astype(np.float)*512/1024

# En segundo lugar, utilizando el ancho y la altura de las bounding boxes, obtenemos
# las coordenadas máximas de las bounding boxes tanto en el eje vertical como en el
# eje horizontal.
df['x_max'] = df['x_min'] + df['w']
df['y_max'] = df['y_min'] + df['h']

A modo de explicar las nuevas columnas que hemos creado, las coordenadas de la esquina inferior derecha son $(\text{x_max}, \text{y_min})$.

Habiendo hecho ya este reescalamiento, a continuación mostramos una imagen.  

In [ ]:
id = 'b6ab77fd7'
Vars = ["x_min", "y_min", "w", "h"]
bboxes = df[df["image_id"] == id].loc[:, Vars]

In [ ]:
fig,ax = plt.subplots(figsize=(18,10))
for i in range(bboxes.shape[0]):
    img = immg.imread(path + id + '.jpg')
    ax.imshow(img,cmap='binary')
    row = bboxes.iloc[i].values
    rectangle = matplotlib.patches.Rectangle((row[0], row[1]), row[2], row[3], linewidth = 2, edgecolor='orangered', facecolor='none')
    ax.text(*row[:2], "wheat", verticalalignment='top', color='cyan', fontsize=13, weight='bold')
    ax.add_patch(rectangle)
plt.show()

# División entrenamiento-validación

En esta sección partimos los datos para conseguir un conjunto de entrenamiento y un conjunto de validación.



In [ ]:
image_ids = df['image_id'].unique()
val_ids = image_ids[-665:]
train_ids = image_ids[:-665]
val_df = df[df['image_id'].isin(val_ids)]
train_df = df[df['image_id'].isin(train_ids)]

# Dataset, DataLoader y Transforms

Pytorch tiene una infraestructura que permite preprocesar y dar un formato conveniente a los datos.

In [ ]:
class WheatDetectionDataset(Dataset):
    def __init__(self, df, IMG_DIR, transform = None):
        self.df = df
        self.img_dir = IMG_DIR
        self.image_ids = self.df["image_id"].unique().tolist()
        self.transform = transform
        
    def __len__(self):
      # Declaramos esta función dentro de la definición de la clase para que cuando se aplique la función len
      # en un objeto de clase WheatDetectionDataset lo que arroje sea la cantidad de ids únicos dentro del data frame.
        return len(self.image_ids)
    
    
    def __getitem__(self, idx):
      # Declaramos los nombres de las columnas que vamos a seleccionar.
        coords = ["x_min", "y_min", "x_max", "y_max"]

      # Declaramos un id específico, el cual es una entrada dentro de la lista de los ids únicos.  
        image_id = self.image_ids[idx]

      # Dado el id específico, filtramos las filas del data frame para recuperar solamente aquellas que coincidan con
      # aquel id. Seleccionamos solamente las columnas que nos interesan, y retornamos este objeto como un
      # numpy.ndarray.   
        boxes = self.df[self.df["image_id"] == image_id].loc[:, coords].values

      # A partir de las coordenadas de las bounding boxes podemos calcular sus áreas.
        area = (boxes[:, 2] - boxes[:, 0])*(boxes[:, 3] - boxes[:, 1]) 

      # Cargamos la imagen en su representación como un numpy.ndarray 1024 x 1024 x 3.
        image = cv2.imread(self.img_dir+image_id+".jpg",cv2.IMREAD_COLOR)

      # Cambiamos el formato de la imagen y le bajamos el brillo.  
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB).astype(np.float32)/255.0

      # Creamos un tensor.    
        labels = torch.ones((boxes.shape[0],), dtype=torch.int64)
        
      # Guardamos los objetos creados dentro de un diccionario llamado target.  
        target = {
                  'boxes': boxes,
                  'labels': labels,
                  'image_id': torch.tensor([idx]),
                  #'image_id': image_id,
                  'area': torch.as_tensor(area, dtype = torch.float32),
                  'iscrowd': torch.zeros((boxes.shape[0],), dtype = torch.int64)
                 }
    
        if self.transform:
            sample = {
                      'image': image,
                      'bboxes': target['boxes'],
                      'labels': labels
                     }
            sample = self.transform(**sample)
            image = sample['image']
            
            target['boxes'] = torch.stack(tuple(map(torch.tensor, zip(*sample['bboxes'])))).permute(1, 0)

        return torch.tensor(image), target, image_id

Tenemos el parámetro opcional transform, el cual nos permite aplicarle una transformación a las imágenes. **Una parte importante de la transformación consiste en convertir la representación de la imagen como un numpy.ndarray en una representación como tensor.**

En el caso de las imágenes de entrenamiento, además de representarlas como tensor también las vamos a rotar.

In [ ]:
def get_train_transform():
    return A.Compose([
        A.Flip(0.5),
        ToTensorV2(p=1.0)
    ], bbox_params={'format': 'pascal_voc', 'label_fields': ['labels']})

def get_val_transform():
    return A.Compose([
        ToTensorV2(p=1.0)
    ], bbox_params={'format': 'pascal_voc', 'label_fields': ['labels']})

Hagamos el ejercicio de ver una imagen de entrenamiento luego de que esta ha sido transformada.

In [ ]:
wheat_train = WheatDetectionDataset(train_df, path, get_train_transform())

In [ ]:
img, tar,_ = wheat_train[0]
bbox = tar['boxes'].numpy()
fig,ax = plt.subplots(figsize=(18,10))
ax.imshow(img.permute(1,2,0).cpu().numpy())
for i in range(len(bbox)):
    box = bbox[i]
    x,y,w,h = box[0], box[1], box[2]-box[0], box[3]-box[1]
    rect = matplotlib.patches.Rectangle((x,y),w,h,linewidth=2,edgecolor='orangered',facecolor='none',)
    ax.text(*box[:2], "wheat", verticalalignment='top', color='cyan', fontsize=13, weight='bold')
    ax.add_patch(rect)
plt.show()

Ahora creamos un objeto de clase WheatDetectionDataset para el conjunto de validación.

In [ ]:
# Nótese que el tercer argumento es la transformación que le aplicamos al conjunto de validación.
# No es la misma transformación que le habíamos aplicado al conjunto de entrenamiento.
wheat_val = WheatDetectionDataset(val_df, path, get_val_transform())

In [ ]:
# Esta es una función que especifica cómo hacemos el batching.
def collate_fn(batch):
    return tuple(zip(*batch))

indices = torch.randperm(len(wheat_train)).tolist()

train_data_loader = DataLoader(
    wheat_train,
    batch_size=8,
    shuffle=False,
    num_workers=4,
    collate_fn=collate_fn
)

valid_data_loader = DataLoader(
    wheat_val,
    batch_size=8,
    shuffle=False,
    num_workers=4,
    collate_fn=collate_fn
)

# Entrenamiento

In [ ]:
num_classes = 2  # 1 class (wheat) + background

model = torchvision.models.detection.fasterrcnn_resnet50_fpn(pretrained=True)

in_features = model.roi_heads.box_predictor.cls_score.in_features

model.roi_heads.box_predictor = FastRCNNPredictor(in_features, num_classes)

In [ ]:
model.to(device)
params = [p for p in model.parameters() if p.requires_grad]
optimizer = torch.optim.SGD(params, lr=0.005, momentum=0.9, weight_decay=0.0005)
lr_scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=3, gamma=0.1)
#lr_scheduler = None

## Averager

In [ ]:
class Averager:
    def __init__(self):
        self.current_total = 0.0
        self.iterations = 0.0

    def send(self, value):
        self.current_total += value
        self.iterations += 1

    @property
    def value(self):
        if self.iterations == 0:
            return 0
        else:
            return 1.0 * self.current_total / self.iterations

    def reset(self):
        self.current_total = 0.0
        self.iterations = 0.0

A continuación aplicamos non-maximum supression. Para mayor referencia visitar el siguiente enlace: https://pytorch.org/vision/stable/ops.html



In [ ]:
def apply_nms(orig_prediction, iou_thresh=0.2):
    

    keep = torchvision.ops.nms(orig_prediction['boxes'], orig_prediction['scores'], iou_thresh)
    
    final_prediction = orig_prediction
    final_prediction['boxes'] = final_prediction['boxes'][keep]
    final_prediction['scores'] = final_prediction['scores'][keep]
    final_prediction['labels'] = final_prediction['labels'][keep]
    
    return final_prediction

In [ ]:
num_epochs = 10

In [ ]:
loss_hist = Averager()
best_epoch = 0
min_loss = sys.maxsize
for epoch in range(num_epochs):
    loss_hist.reset()
    tk = tqdm(train_data_loader)
    model.train();
    for images, targets, image_ids in tk:
        
        images = list(image.to(device) for image in images)
        targets = [{k: v.to(device) for k, v in t.items()} for t in targets]

        loss_dict = model(images, targets)

        losses = sum(loss for loss in loss_dict.values())
        loss_value = losses.item()

        loss_hist.send(loss_value)

        optimizer.zero_grad()
        losses.backward()
        optimizer.step()
        
        tk.set_postfix(train_loss=loss_value)
    tk.close()
    
    # update the learning rate
    if lr_scheduler is not None:
        lr_scheduler.step()

    print(f"Epoch #{epoch} loss: {loss_hist.value}") 
    
    if loss_hist.value<min_loss:
        print("Better model found at epoch {0} with {1:0.5f} loss value".format(epoch,loss_hist.value))
        torch.save(model.state_dict(), f"model_state_epoch_{epoch}.pth")
        min_loss = loss_hist.value
        best_epoch = epoch
    #validation 
    model.eval();
    with torch.no_grad():
        tk = tqdm(valid_data_loader)
        for images, targets, image_ids in tk:
        
            images = list(image.to(device) for image in images)
            targets = [{k: v.to(device) for k, v in t.items()} for t in targets]
            val_output = model(images)
            val_output = [{k: v.to('cpu') for k, v in t.items()} for t in val_output]
            IOU = []
            for j in range(len(val_output)):
                val_out = apply_nms(val_output[j])
                a,b = val_out['boxes'].cpu().detach(), targets[j]['boxes'].cpu().detach()
                chk = torchvision.ops.box_iou(a,b)
                res = np.nanmean(chk.sum(axis=1)/(chk>0).sum(axis=1))
                IOU.append(res)
            tk.set_postfix(IoU=np.mean(IOU))
        tk.close()
        
model.load_state_dict(torch.load(f"./model_state_epoch_{best_epoch}.pth"));

In [ ]:
model.load_state_dict(torch.load(f"./model_state_epoch_{best_epoch}.pth"));

# Validación y predicción

In [ ]:
img,target,_ = wheat_val[50]

model.eval()
with torch.no_grad():
    prediction = model([img.to(device)])[0]
    
print('predicted #boxes: ', len(prediction['boxes']))
print('real #boxes: ', len(target['boxes']))

# Ground truths

In [ ]:
bbox = target['boxes'].numpy()
fig,ax = plt.subplots(1,figsize=(18,10))
ax.imshow(img.permute(1,2,0).cpu().numpy())
for i in range(len(bbox)):
    box = bbox[i]
    x,y,w,h = box[0], box[1], box[2]-box[0], box[3]-box[1]
    rect = matplotlib.patches.Rectangle((x,y),w,h,linewidth=2,edgecolor='r',facecolor='none',)
    ax.text(*box[:2], "wheat", verticalalignment='top', color='red', fontsize=13, weight='bold')
    ax.add_patch(rect)
plt.show()

In [ ]:
def plot_valid(img,prediction,nms=True,detect_thresh=0.5):
    fig,ax = plt.subplots(figsize=(18,10))
    val_img = img.permute(1,2,0).cpu().numpy()
    ax.imshow(val_img)
    nms_prediction = apply_nms(prediction, iou_thresh=0.2) if nms else prediction
    val_scores = nms_prediction['scores'].cpu().detach().numpy()
    bbox = nms_prediction['boxes'].cpu().detach().numpy()
    for i in range(len(bbox)):
        if val_scores[i]>=detect_thresh:
            box = bbox[i]
            x,y,w,h = box[0], box[1], box[2]-box[0], box[3]-box[1]
            rect = matplotlib.patches.Rectangle((x,y),w,h,linewidth=2 ,edgecolor='r',facecolor='none',)
            ax.text(*box[:2], "wheat {0:.3f}".format(val_scores[i]), verticalalignment='top', color='white', fontsize=12, weight='bold')
            ax.add_patch(rect)
    plt.show()

# Predicciones en el conjunto de validación

In [ ]:
plot_valid(img,prediction)

# Predicciones en el conjunto de test

In [ ]:
submission = pd.read_csv('../input/global-wheat-detection/sample_submission.csv')

In [ ]:
class TestDataset(object):
    def __init__(self, df, IMG_DIR, transforms=None):
        # select only those classes that have boxes
        
        self.df = df
        self.img_dir = IMG_DIR
        self.transforms = transforms
        self.image_ids = self.df['image_id'].tolist()
        
    def __len__(self):
        return len(self.image_ids)
    
    def __getitem__(self, idx):
        
        image_id = self.image_ids[idx]
        image = cv2.imread(self.img_dir+image_id+".jpg",cv2.IMREAD_COLOR)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB).astype(np.float32)
        
        
        if self.transforms:
            sample = {
                'image': image,
            }
            sample = self.transforms(**sample)
            image = sample['image']

        return image, image_id

In [ ]:
def get_test_transform(IMG_SIZE=(512,512)):
    return A.Compose([
         A.Normalize(mean=(0, 0, 0), std=(1, 1, 1), max_pixel_value=255.0, p=1.0),
        A.Resize(*IMG_SIZE),
        ToTensorV2(p=1.0)
    ])

In [ ]:
# Acá ponemos la ruta del conjunto test.
test_img_dir = '../input/global-wheat-detection/test/'

In [ ]:
IMG_SIZE = (512,512)

In [ ]:
test_dataset = TestDataset(submission, test_img_dir ,get_test_transform(IMG_SIZE))

In [ ]:
for j in range(submission.shape[0]):
    img,_ = test_dataset[j]
    # put the model in evaluation mode
    model.eval()
    with torch.no_grad():
        prediction = model([img.to(device)])[0]
    plot_valid(img,prediction)